In [1]:
using DrWatson
@quickactivate "Attitude Control"

In [2]:
using CBinding

In [3]:
LD_PATH = [
    "../build/",
]

INC_PATH = [
    "../lib/libutil/include/",
    "../lib/libadcs/include/",
    "../lib/libadcs/lib/libadcs_client/include",
]

LD_PATH = map(p -> "-L"*p, LD_PATH)
INC_PATH = map(i -> "-I"*i, INC_PATH)
LIB = "shadcs";

In [ ]:
## C environment
c`$(INC_PATH) $(LD_PATH) -l$(LIB)`

In [ ]:
# Define stdint sizes
const c"int8_t"  = Int8
const c"uint8_t"  = UInt8

const c"int16_t"  = Int16
const c"uint16_t"  = UInt16

const c"int32_t"  = Int32
const c"uint32_t" = UInt32

const c"int64_t"  = Int64
const c"uint64_t" = UInt64;

In [ ]:
# Setup includes

# Don't konw why these are needed, but for some reason, they are :S
c"""
#include <gs/adcs/param/host/sensor_fss.h>

#include <gs/adcs/param/host/gnc_common.h>
#include <gs/adcs/param/host/gnc_ads.h>
#include <gs/adcs/param/host/gnc_ctrl_pointing.h>
#include <gs/adcs/param/host/gnc_ctrl_spin.h>
"""

c"""
#include <gs/adcs/adcs_param_types.h>
#include <gs/adcs/adcs_telem_types.h>

#include <gs/adcs/acs/controller/gs_adcs_wheel.h>
""";

In [ ]:
gnc = Libc.malloc(c"gs_adcs_config_gnc_t"(
        common=(
            inertia=(1,2,3),
        )
    )
)

periph = Libc.malloc(c"gs_adcs_config_periph_t"())
fss = Libc.malloc(c"gs_adcs_sensor_fss_memory_t"())

conf = Libc.malloc(c"gs_adcs_config_t"(
        gnc=gnc, periph=periph
    )
)

ephem = Libc.malloc(c"gs_adcs_ephem_t"())

state_est = Libc.malloc(c"GS_ADCS_UKF_Data_t"(
        EstimatedState=(
            1.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01
        )
    )
)

act_data = Libc.malloc(c"gs_adcs_actuatordata_t"())
ctrl = Libc.malloc(c"gs_adcs_ctrl_t"(ref_q=(q4=1.0,)))

# Get handle to function. 
# Could also do c"gs_adcs_wheel_ctrl"(conf, ephem, state_est, act_data, ctr)
smc = c"gs_adcs_wheel_ctrl"[] 

In [ ]:
smc(conf, ephem, state_est, act_data, ctrl)
ctrl[]

In [ ]:
ctrl

In [ ]:
tmp = c"gs_adcs_ctrl_t"()